In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Mar  5 06:35:12 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    50W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


In [4]:
import torch

!pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster  --y
!pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

Found existing installation: torch-scatter 2.1.0+pt113cu116
Uninstalling torch-scatter-2.1.0+pt113cu116:
  Successfully uninstalled torch-scatter-2.1.0+pt113cu116
Found existing installation: torch-sparse 0.6.16+pt113cu116
Uninstalling torch-sparse-0.6.16+pt113cu116:
  Successfully uninstalled torch-sparse-0.6.16+pt113cu116
Found existing installation: torch-geometric 2.3.0
Uninstalling torch-geometric-2.3.0:
  Successfully uninstalled torch-geometric-2.3.0
Found existing installation: torch-cluster 1.6.0+pt113cu116
Uninstalling torch-cluster-1.6.0+pt113cu116:
  Successfully uninstalled torch-cluster-1.6.0+pt113cu116
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.1+cu116.html
  Using cached https://data.pyg.org/whl/torch-1.13.0%2Bcu116/torch_scatter-2.1.0%2Bpt113cu116-cp38-cp38-linux_x86_64.whl (9.4 MB)
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wh

In [5]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import pickle
import torch
import transformers
from transformers import BertTokenizer, BertModel

from transformers import AutoTokenizer, AutoModel, AutoConfig


# from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv, GATConv
from torch_geometric.nn import global_mean_pool
from torch_geometric.data import InMemoryDataset, download_url
from torch_geometric.data import Data

from torch_geometric.loader import DataLoader

import torch.nn.functional as F
import numpy as np

import math
import pandas as pd
import pip
import re

import os.path as osp

import torch
# from torch_geometric.data import Dataset, download_url

from torch.utils.data import Dataset

import random

random.seed(123)

torch.manual_seed(123)
np.random.seed(123)
torch.cuda.manual_seed_all(123)
from sklearn.metrics import jaccard_score, f1_score, accuracy_score, recall_score, precision_score, confusion_matrix


In [7]:
with open("/content/drive/MyDrive/Colab Notebooks/pakdd_baseline/pakdd_model/full_dialog/two_disease/final_combined_all_dialog.p", "rb") as f:
    final_dialog_sheet = pickle.load(f)

In [8]:
final_dialog_sheet

[{'dialog': "[PAT] : Doctor, I see Spots or clouds in my vision. I can't see anything clearly. I feel helpless. [SEP] [DOC] : I understand how you feel. I need to ask a few questions to diagnose your disease. Have you got Diminished vision? [SEP] [PAT] : True, I am also dealing with Diminished vision. [SEP] [DOC] : Do you have Symptoms of eye? [SEP] [PAT] : Yes, I have got Symptoms of eye. [SEP] [DOC] : Have you felt Pain in eye? [SEP] [PAT] : Indeed, I am suffering from Pain in eye. [SEP] [DOC] : Your symptoms indicate that you have [MASK].",
  'label': 'central retinal artery or vein occlusion'},
 {'dialog': "[PAT] : Doctor, I have Shoulder pain. I feel terrible. Would you please tell me what could be its cause? [SEP] [DOC] :  I can feel the pain you feel. Let me ask you some questions. Do you have Back pain? [SEP] [PAT] : Literally, I am suffering from Back pain. [SEP] [DOC] : Do you have Low back pain? [SEP] [PAT] : Yes, I am dealing with Low back pain. [SEP] [DOC] : Are you also d

In [9]:

total_dialog_sheet = final_dialog_sheet
len(total_dialog_sheet)

1367

In [10]:
with open("/content/drive/MyDrive/Colab Notebooks/pakdd_baseline/pakdd_model/self_report/two_disease/final_combined_all_self_report.p", "rb") as f:
  self_report = pickle.load(f)

In [11]:
self_report

[{'first_utterance': "[SR_START] [PAT] : Doctor, I see Spots or clouds in my vision. I can't see anything clearly. I feel helpless. [SR_END]",
  'label': 'central retinal artery or vein occlusion'},
 {'first_utterance': '[SR_START] [PAT] : Doctor, I have Shoulder pain. I feel terrible. Would you please tell me what could be its cause? [SR_END]',
  'label': 'degenerative disc disease'},
 {'first_utterance': '[SR_START] [PAT] : Hi! Doctor, I am suffering from Foreign body sensation in eye. Would you please tell me what could be its cause? [SR_END]',
  'label': 'diabetic retinopathy'},
 {'first_utterance': "[SR_START] [PAT] : Doctor, I am dealing with Low back pain. I can't sleep well because of this, and in the daytime feels lethargic. Please help me.  [SR_END]",
  'label': 'chronic back pain'},
 {'first_utterance': '[SR_START] [PAT] : Doctor, I have been feeling Wrist pain since the previous monday. What is wrong with me? [SR_END]',
  'label': 'air embolism'},
 {'first_utterance': '[SR_

In [12]:
len(self_report)

1367

In [13]:
self_report

[{'first_utterance': "[SR_START] [PAT] : Doctor, I see Spots or clouds in my vision. I can't see anything clearly. I feel helpless. [SR_END]",
  'label': 'central retinal artery or vein occlusion'},
 {'first_utterance': '[SR_START] [PAT] : Doctor, I have Shoulder pain. I feel terrible. Would you please tell me what could be its cause? [SR_END]',
  'label': 'degenerative disc disease'},
 {'first_utterance': '[SR_START] [PAT] : Hi! Doctor, I am suffering from Foreign body sensation in eye. Would you please tell me what could be its cause? [SR_END]',
  'label': 'diabetic retinopathy'},
 {'first_utterance': "[SR_START] [PAT] : Doctor, I am dealing with Low back pain. I can't sleep well because of this, and in the daytime feels lethargic. Please help me.  [SR_END]",
  'label': 'chronic back pain'},
 {'first_utterance': '[SR_START] [PAT] : Doctor, I have been feeling Wrist pain since the previous monday. What is wrong with me? [SR_END]',
  'label': 'air embolism'},
 {'first_utterance': '[SR_

In [14]:
total_dialog_sheet[0]

{'dialog': "[PAT] : Doctor, I see Spots or clouds in my vision. I can't see anything clearly. I feel helpless. [SEP] [DOC] : I understand how you feel. I need to ask a few questions to diagnose your disease. Have you got Diminished vision? [SEP] [PAT] : True, I am also dealing with Diminished vision. [SEP] [DOC] : Do you have Symptoms of eye? [SEP] [PAT] : Yes, I have got Symptoms of eye. [SEP] [DOC] : Have you felt Pain in eye? [SEP] [PAT] : Indeed, I am suffering from Pain in eye. [SEP] [DOC] : Your symptoms indicate that you have [MASK].",
 'label': 'central retinal artery or vein occlusion'}

In [15]:
total_sep_dialog = []

for i in range(len(total_dialog_sheet)):
  temp_dlog = total_dialog_sheet[i]['dialog']
  total_sep_dialog.append(temp_dlog)

In [16]:
total_sep_dialog[0]

"[PAT] : Doctor, I see Spots or clouds in my vision. I can't see anything clearly. I feel helpless. [SEP] [DOC] : I understand how you feel. I need to ask a few questions to diagnose your disease. Have you got Diminished vision? [SEP] [PAT] : True, I am also dealing with Diminished vision. [SEP] [DOC] : Do you have Symptoms of eye? [SEP] [PAT] : Yes, I have got Symptoms of eye. [SEP] [DOC] : Have you felt Pain in eye? [SEP] [PAT] : Indeed, I am suffering from Pain in eye. [SEP] [DOC] : Your symptoms indicate that you have [MASK]."

In [17]:
len(total_sep_dialog)

1367

In [18]:
self_report[0]

{'first_utterance': "[SR_START] [PAT] : Doctor, I see Spots or clouds in my vision. I can't see anything clearly. I feel helpless. [SR_END]",
 'label': 'central retinal artery or vein occlusion'}

In [19]:
self_report_utterance = []

for i in range(len(self_report)):
  temp_utterance = self_report[i]['first_utterance']
  self_report_utterance.append(temp_utterance)


In [20]:
self_report_utterance[0]

"[SR_START] [PAT] : Doctor, I see Spots or clouds in my vision. I can't see anything clearly. I feel helpless. [SR_END]"

In [21]:
len(self_report_utterance)

1367

In [22]:
self_report_utterance[237]

"[SR_START] [PAT] : Hello doctor, I have experienced Leg pain. Would you please tell me its cause? I can't sleep all night due to this. Please help me doctor. [SR_END]"

In [23]:
total_sep_dialog[237]

"[PAT] : Hello doctor, I have experienced Leg pain. Would you please tell me its cause? I can't sleep all night due to this. Please help me doctor. [SEP] [DOC] : OK, I wish you didn't have to go through that pain. Leg pain is a common symptom. What matters most is the symptoms associated with it. Have you gone through Seizures? [SEP] [PAT] : True, sometimes I have got Seizures. [SEP] [DOC] : Have you experienced Disturbance of memory? [SEP] [PAT] : Literally, I am suffering from it. [SEP] [DOC] : Do you have Problems with movement? [SEP] [PAT] : Exactly, I have got Problems with movement. [SEP] [DOC] : Alright, you may have [MASK]."

In [24]:
class MyOwnDataset_complete(InMemoryDataset):
    def __init__(self, root, transform = None, pre_transform = None, pre_filter = None):
        super().__init__(root, transform, pre_transform, pre_filter)
        self.data, self.slices = torch.load(self.processed_paths[0])
        
    @property
    def processed_file_names(self):
        return ['data_complete_1367.pt']

In [25]:
dataset_joint_kg = MyOwnDataset_complete(root = '/content/drive/MyDrive/Colab Notebooks/pakdd_baseline/pakdd_model/full_dialog/two_disease/joint_graph/')
dataset_joint_kg

MyOwnDataset_complete(1367)

In [26]:
data_joint_kg = []
for i in dataset_joint_kg:
    data_joint_kg.append(i)

In [27]:
data_joint_kg

[Data(x=[11, 768], edge_index=[2, 24], edge_attr=[24], y=[1]),
 Data(x=[16, 768], edge_index=[2, 48], edge_attr=[48], y=[1]),
 Data(x=[4, 768], edge_index=[2, 3], edge_attr=[3], y=[1]),
 Data(x=[9, 768], edge_index=[2, 15], edge_attr=[15], y=[1]),
 Data(x=[13, 768], edge_index=[2, 120], edge_attr=[120], y=[1]),
 Data(x=[7, 768], edge_index=[2, 8], edge_attr=[8], y=[1]),
 Data(x=[16, 768], edge_index=[2, 35], edge_attr=[35], y=[1]),
 Data(x=[10, 768], edge_index=[2, 15], edge_attr=[15], y=[1]),
 Data(x=[10, 768], edge_index=[2, 32], edge_attr=[32], y=[1]),
 Data(x=[10, 768], edge_index=[2, 15], edge_attr=[15], y=[1]),
 Data(x=[13, 768], edge_index=[2, 46], edge_attr=[46], y=[1]),
 Data(x=[17, 768], edge_index=[2, 48], edge_attr=[48], y=[1]),
 Data(x=[13, 768], edge_index=[2, 120], edge_attr=[120], y=[1]),
 Data(x=[8, 768], edge_index=[2, 15], edge_attr=[15], y=[1]),
 Data(x=[8, 768], edge_index=[2, 14], edge_attr=[14], y=[1]),
 Data(x=[16, 768], edge_index=[2, 48], edge_attr=[48], y=[1]

In [28]:
len(data_joint_kg)

1367

In [29]:
tokenizer  = AutoTokenizer.from_pretrained("cambridgeltl/SapBERT-from-PubMedBERT-fulltext")
bert_model = AutoModel.from_pretrained("cambridgeltl/SapBERT-from-PubMedBERT-fulltext")
bert_model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [30]:
print(tokenizer.all_special_tokens)
print(tokenizer.all_special_ids)
p = {
        'additional_special_tokens' : ['[DOC]', '[PAT]', '[SR_START]', '[SR_END]']
    }

tokenizer.add_special_tokens(p)
print(tokenizer.convert_tokens_to_ids('[DOC]'))
print(tokenizer.convert_tokens_to_ids('[PAT]'))
print(tokenizer.convert_tokens_to_ids('[SR_START]'))
print(tokenizer.convert_tokens_to_ids('[SR_END]'))
print(len(tokenizer))
print(bert_model.resize_token_embeddings(len(tokenizer)))
print(tokenizer.all_special_tokens)
print(tokenizer.all_special_ids)

['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']
[1, 3, 0, 2, 4]
30522
30523
30524
30525
30526
Embedding(30526, 768)
['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]', '[DOC]', '[PAT]', '[SR_START]', '[SR_END]']
[1, 3, 0, 2, 4, 30522, 30523, 30524, 30525]


In [31]:
len(data_joint_kg)

1367

In [32]:
data_combined = []

for i in range(len(data_joint_kg)):
    temp_list = []
    temp_list.append(data_joint_kg[i])
#     temp_list.append(input_ids[i,:])
#     temp_list.append(attention_mask[i,:])
    temp_list.append(total_sep_dialog[i])
    temp_list.append(self_report_utterance[i])
    data_combined.append(temp_list)

In [33]:
len(data_combined)

1367

In [34]:
len(data_combined[0])

3

In [35]:
data_combined[0]

[Data(x=[11, 768], edge_index=[2, 24], edge_attr=[24], y=[1]),
 "[PAT] : Doctor, I see Spots or clouds in my vision. I can't see anything clearly. I feel helpless. [SEP] [DOC] : I understand how you feel. I need to ask a few questions to diagnose your disease. Have you got Diminished vision? [SEP] [PAT] : True, I am also dealing with Diminished vision. [SEP] [DOC] : Do you have Symptoms of eye? [SEP] [PAT] : Yes, I have got Symptoms of eye. [SEP] [DOC] : Have you felt Pain in eye? [SEP] [PAT] : Indeed, I am suffering from Pain in eye. [SEP] [DOC] : Your symptoms indicate that you have [MASK].",
 "[SR_START] [PAT] : Doctor, I see Spots or clouds in my vision. I can't see anything clearly. I feel helpless. [SR_END]"]

In [36]:
data_combined[0][1]

"[PAT] : Doctor, I see Spots or clouds in my vision. I can't see anything clearly. I feel helpless. [SEP] [DOC] : I understand how you feel. I need to ask a few questions to diagnose your disease. Have you got Diminished vision? [SEP] [PAT] : True, I am also dealing with Diminished vision. [SEP] [DOC] : Do you have Symptoms of eye? [SEP] [PAT] : Yes, I have got Symptoms of eye. [SEP] [DOC] : Have you felt Pain in eye? [SEP] [PAT] : Indeed, I am suffering from Pain in eye. [SEP] [DOC] : Your symptoms indicate that you have [MASK]."

In [37]:
random.shuffle(data_combined)

In [38]:
joint_data_after_shuffling = []
for i in data_combined:
    joint_graph = i[0]
#     print(joint_graph)
#     print("\n")
    joint_data_after_shuffling.append(joint_graph)

In [39]:
len(joint_data_after_shuffling)

1367

In [40]:
joint_data_after_shuffling[0]

Data(x=[4, 768], edge_index=[2, 3], edge_attr=[3], y=[1])

In [41]:
joint_data_after_shuffling[0].y

tensor([53.])

In [42]:
joint_data_after_shuffling_train = joint_data_after_shuffling[:957]
len(joint_data_after_shuffling_train)

957

In [43]:
joint_data_after_shuffling_valid = joint_data_after_shuffling[957:1094]
len(joint_data_after_shuffling_valid)

137

In [44]:
joint_data_after_shuffling_test = joint_data_after_shuffling[1094:]
len(joint_data_after_shuffling_test)

273

In [45]:
dialog_after_shuffling = []
for i in data_combined:
    dlog = i[1]
    print(dlog)
    dialog_after_shuffling.append(dlog)
    print()
    

[PAT] : Hello! Doctor. I have got Skin lesion. I am a little worried about it. [SEP] [DOC] : Per your symptom, you may have [MASK]. 

[PAT] : Doctor, I have got Fatigue. I don't feel energetic in any condition, help me. [SEP] [DOC] : I get where you are coming from. Do you have Pain or soreness of breast? [SEP] [PAT] : Yes Doctor. [SEP] [DOC] : Are you suffering from Excessive urination at night? [SEP] [PAT] : Yes. [SEP] [DOC] : You might have [MASK].

[PAT] : Hello doctor, I am suffering from Fatigue. [SEP] [DOC] : Alright, have you felt Dizziness? [SEP] [PAT] : True, I have Dizziness too. [SEP] [DOC] : Have you got Foot or toe swelling? [SEP] [PAT] : Please, see Foot or toe swelling. [SEP] [DOC] : Your symptoms indicate that you have [MASK].

[PAT] : Hi Doctor, why am I having Fever. I have to get well before next week. I have my final exams. [SEP] [DOC] : I can see how urgent it is for you to get well soon. Let me see. Do you have Skin rash? [SEP] [PAT] : Not sure. Please confirm it

In [46]:
len(dialog_after_shuffling)

1367

In [47]:
dialog_after_shuffling[0]

'[PAT] : Hello! Doctor. I have got Skin lesion. I am a little worried about it. [SEP] [DOC] : Per your symptom, you may have [MASK]. '

In [48]:
dialog_after_shuffling_train = dialog_after_shuffling[:957]
len(dialog_after_shuffling_train)

957

In [49]:
dialog_after_shuffling_valid = dialog_after_shuffling[957:1094]
len(dialog_after_shuffling_valid)

137

In [50]:
dialog_after_shuffling_test = dialog_after_shuffling[1094:]
len(dialog_after_shuffling_test)

273

In [51]:
enc_train = tokenizer(dialog_after_shuffling_train, padding='max_length', max_length=512, truncation=True, return_tensors = 'pt')
enc_train

{'input_ids': tensor([[    2, 30523,    30,  ...,     0,     0,     0],
        [    2, 30523,    30,  ...,     0,     0,     0],
        [    2, 30523,    30,  ...,     0,     0,     0],
        ...,
        [    2, 30523,    30,  ...,     0,     0,     0],
        [    2, 30523,    30,  ...,     0,     0,     0],
        [    2, 30523,    30,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [52]:
enc_valid = tokenizer(dialog_after_shuffling_valid, padding='max_length', max_length=512, truncation=True, return_tensors= 'pt')
enc_valid

{'input_ids': tensor([[    2, 30523,    30,  ...,     0,     0,     0],
        [    2, 30523,    30,  ...,     0,     0,     0],
        [    2, 30523,    30,  ...,     0,     0,     0],
        ...,
        [    2, 30523,    30,  ...,     0,     0,     0],
        [    2, 30523,    30,  ...,     0,     0,     0],
        [    2, 30523,    30,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [53]:
enc_test = tokenizer(dialog_after_shuffling_test, padding='max_length', max_length=512, truncation=True, return_tensors='pt')
enc_test

{'input_ids': tensor([[    2, 30523,    30,  ...,     0,     0,     0],
        [    2, 30523,    30,  ...,     0,     0,     0],
        [    2, 30523,    30,  ...,     0,     0,     0],
        ...,
        [    2, 30523,    30,  ...,     0,     0,     0],
        [    2, 30523,    30,  ...,     0,     0,     0],
        [    2, 30523,    30,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [54]:
self_report_after_shuffling = []
for i in data_combined:
    sr = i[2]
    print(sr)
    self_report_after_shuffling.append(sr)
    print()

[SR_START] [PAT] : Hello! Doctor. I have got Skin lesion. I am a little worried about it. [SR_END]

[SR_START] [PAT] : Doctor, I have got Fatigue. I don't feel energetic in any condition, help me. [SR_END]

[SR_START] [PAT] : Hello doctor, I am suffering from Fatigue. [SR_END]

[SR_START] [PAT] : Hi Doctor, why am I having Fever. I have to get well before next week. I have my final exams. [SR_END]

[SR_START] [PAT] : Hello doctor, I am struggling with Shortness of breath. I donot want to make this severe. Please help. [SR_END]

[SR_START] [PAT] : Hi doctor. I have got Leg pain. Would you please tell me its cause? I can't walk. I am unable to move even to the outside of our house. [SR_END]

[SR_START] [PAT] : I am ill with Dizziness. I cannot concentrate while studying. [SR_END]

[SR_START] [PAT] : Doctor, I am ill with Weakness. I need to gain my strength before my exam. [SR_END]

[SR_START] [PAT] : Doctor, I am dealing with Abnormal involuntary movements. [SR_END]

[SR_START] [PAT] : 

In [55]:
self_report_after_shuffling

['[SR_START] [PAT] : Hello! Doctor. I have got Skin lesion. I am a little worried about it. [SR_END]',
 "[SR_START] [PAT] : Doctor, I have got Fatigue. I don't feel energetic in any condition, help me. [SR_END]",
 '[SR_START] [PAT] : Hello doctor, I am suffering from Fatigue. [SR_END]',
 '[SR_START] [PAT] : Hi Doctor, why am I having Fever. I have to get well before next week. I have my final exams. [SR_END]',
 '[SR_START] [PAT] : Hello doctor, I am struggling with Shortness of breath. I donot want to make this severe. Please help. [SR_END]',
 "[SR_START] [PAT] : Hi doctor. I have got Leg pain. Would you please tell me its cause? I can't walk. I am unable to move even to the outside of our house. [SR_END]",
 '[SR_START] [PAT] : I am ill with Dizziness. I cannot concentrate while studying. [SR_END]',
 '[SR_START] [PAT] : Doctor, I am ill with Weakness. I need to gain my strength before my exam. [SR_END]',
 '[SR_START] [PAT] : Doctor, I am dealing with Abnormal involuntary movements. [SR

In [56]:
self_report_after_shuffling_train = self_report_after_shuffling[:957]
print(len(self_report_after_shuffling_train))
self_report_after_shuffling_train

957


['[SR_START] [PAT] : Hello! Doctor. I have got Skin lesion. I am a little worried about it. [SR_END]',
 "[SR_START] [PAT] : Doctor, I have got Fatigue. I don't feel energetic in any condition, help me. [SR_END]",
 '[SR_START] [PAT] : Hello doctor, I am suffering from Fatigue. [SR_END]',
 '[SR_START] [PAT] : Hi Doctor, why am I having Fever. I have to get well before next week. I have my final exams. [SR_END]',
 '[SR_START] [PAT] : Hello doctor, I am struggling with Shortness of breath. I donot want to make this severe. Please help. [SR_END]',
 "[SR_START] [PAT] : Hi doctor. I have got Leg pain. Would you please tell me its cause? I can't walk. I am unable to move even to the outside of our house. [SR_END]",
 '[SR_START] [PAT] : I am ill with Dizziness. I cannot concentrate while studying. [SR_END]',
 '[SR_START] [PAT] : Doctor, I am ill with Weakness. I need to gain my strength before my exam. [SR_END]',
 '[SR_START] [PAT] : Doctor, I am dealing with Abnormal involuntary movements. [SR

In [57]:
self_report_after_shuffling_valid = self_report_after_shuffling[957:1094]
print(len(self_report_after_shuffling_valid))
self_report_after_shuffling_valid

137


['[SR_START] [PAT] : I have been experiencing disturbance of memory. Recently, I have noticed that I am asking the same questions repeatedly to the same person. Also, I tend to forget common words while talking. Is it something serious doctor? I am worried. [SR_END]',
 '[SR_START] [PAT] : Hi doctor, I am struggling with Excessive urination at night. I am really concerned about it. I feel annoyed because of it. Please help. [SR_END]',
 '[SR_START] [PAT] : Doctor, I have Difficulty in swallowing. I have not been able to eat or drink anything because of the pain. [SR_END]',
 "[SR_START] [PAT] : Good Morning Doctor. Please check why I have Depressive or psychotic symptoms. I feel like I want to end my life, what's wrong with me? [SR_END]",
 '[SR_START] [PAT] : Hello doctor, I am suffering from Wrist pain. It is making me difficult to grip object. Help me out. [SR_END]',
 '[SR_START] [PAT] : Hi Doctor, I am dealing with Joint stiffness or tightness. It is making me difficult to move properl

In [58]:
self_report_after_shuffling_test = self_report_after_shuffling[1094:]
print(len(self_report_after_shuffling_test))
self_report_after_shuffling_test

273


['[SR_START] [PAT] : Doctor, I am having some issues. See, Skin rash [SR_END]',
 '[SR_START] [PAT] : Hi Doctor, I am suffering from Shortness of breath. Due to it I am unable to breath properly, help me. [SR_END]',
 '[SR_START] [PAT] : Hello doctor, I am feeling Foot or toe pain. I am unable to run or walk fast. Can you please help me. [SR_END]',
 '[SR_START] [PAT] : Doctor, I have got Knee pain. I cannot miss my hike this week.Please help. [SR_END]',
 '[SR_START] [PAT] : Doctor, I am suffering from Anxiety and nervousness. It is making me tensed in every situation. [SR_END]',
 '[SR_START] [PAT] : Hi doctor, I have Leg pain. I have to participate in a marathon on this Thrusday. I am afraid wether I could participate or not. [SR_END]',
 '[SR_START] [PAT] : Doctor, I am having Allergic reaction. [SR_END]',
 '[SR_START] [PAT] : Doctor, I am going through Infertility. Please, let me know its cause. I am afraid I will never have a child. [SR_END]',
 '[SR_START] [PAT] : Hi! I have got Mass o

In [59]:
enc_self_report_train = tokenizer(self_report_after_shuffling_train, padding = 'max_length', max_length = 512, truncation=True, return_tensors= 'pt' )
enc_self_report_train

{'input_ids': tensor([[    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0],
        ...,
        [    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [60]:
enc_self_report_valid = tokenizer(self_report_after_shuffling_valid, padding = 'max_length', max_length = 512, truncation = True, return_tensors = 'pt')
enc_self_report_valid

{'input_ids': tensor([[    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0],
        ...,
        [    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [61]:
enc_self_report_test = tokenizer(self_report_after_shuffling_test, padding = 'max_length', max_length = 512, truncation = True, return_tensors = 'pt')
enc_self_report_test

{'input_ids': tensor([[    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0],
        ...,
        [    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [62]:
# x1 - Joint Graph data
# x2 - input ids dialog
# x3 - attention mask dialog
# x4 - input ids self report
# x5 - attention mask self report
class Dataset_Knowledge_Linear_SelfReport(Dataset):
    def __init__(self, x1, x2, x3, x4, x5):
        self.x1 = x1
        self.x2 = x2
        self.x3 = x3
        self.x4 = x4
        self.x5 = x5
    
    def __len__(self):
        return len(self.x1)
    
    def __getitem__(self, idx):
        x1_idx = self.x1[idx]
        x2_idx = self.x2[idx]
        x3_idx = self.x3[idx]
        x4_idx = self.x4[idx]
        x5_idx = self.x5[idx]
        
        return x1_idx, x2_idx, x3_idx, x4_idx, x5_idx
        

In [63]:
train_loader = DataLoader(Dataset_Knowledge_Linear_SelfReport(joint_data_after_shuffling_train, enc_train['input_ids'], enc_train['attention_mask'], enc_self_report_train['input_ids'], enc_self_report_train['attention_mask']), batch_size = 16, shuffle = True)
train_loader




In [64]:
valid_loader = DataLoader(Dataset_Knowledge_Linear_SelfReport(joint_data_after_shuffling_valid, enc_valid['input_ids'], enc_valid['attention_mask'], enc_self_report_valid['input_ids'], enc_self_report_valid['attention_mask']), batch_size = 16, shuffle=False)
valid_loader



In [65]:
test_loader = DataLoader(Dataset_Knowledge_Linear_SelfReport(joint_data_after_shuffling_test, enc_test['input_ids'], enc_test['attention_mask'], enc_self_report_test['input_ids'], enc_self_report_test['attention_mask']), batch_size=16, shuffle=False)
test_loader



In [66]:
for step,data in enumerate(train_loader):
    print("Step : ", step)
    print("Data : ", data)
    print()

Step :  0
Data :  [DataBatch(x=[152, 768], edge_index=[2, 323], edge_attr=[323], y=[16], batch=[152], ptr=[17]), tensor([[    2, 30523,    30,  ...,     0,     0,     0],
        [    2, 30523,    30,  ...,     0,     0,     0],
        [    2, 30523,    30,  ...,     0,     0,     0],
        ...,
        [    2, 30523,    30,  ...,     0,     0,     0],
        [    2, 30523,    30,  ...,     0,     0,     0],
        [    2, 30523,    30,  ...,     0,     0,     0]]), tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), tensor([[    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0],
        ...,
        [    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0],
  

In [67]:
for step,data in enumerate(valid_loader):
    print("Step : ", step)
    print("Data : ", data)
    print("Label : ", data[0].y)
    print()

Step :  0
Data :  [DataBatch(x=[150, 768], edge_index=[2, 338], edge_attr=[338], y=[16], batch=[150], ptr=[17]), tensor([[    2, 30523,    30,  ...,     0,     0,     0],
        [    2, 30523,    30,  ...,     0,     0,     0],
        [    2, 30523,    30,  ...,     0,     0,     0],
        ...,
        [    2, 30523,    30,  ...,     0,     0,     0],
        [    2, 30523,    30,  ...,     0,     0,     0],
        [    2, 30523,    30,  ...,     0,     0,     0]]), tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), tensor([[    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0],
        ...,
        [    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0],
  

In [68]:
for step,data in enumerate(test_loader):
    print("Step : ", step)
    print("Data : ", data)
    print()

Step :  0
Data :  [DataBatch(x=[158, 768], edge_index=[2, 373], edge_attr=[373], y=[16], batch=[158], ptr=[17]), tensor([[    2, 30523,    30,  ...,     0,     0,     0],
        [    2, 30523,    30,  ...,     0,     0,     0],
        [    2, 30523,    30,  ...,     0,     0,     0],
        ...,
        [    2, 30523,    30,  ...,     0,     0,     0],
        [    2, 30523,    30,  ...,     0,     0,     0],
        [    2, 30523,    30,  ...,     0,     0,     0]]), tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), tensor([[    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0],
        ...,
        [    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0],
  

In [69]:
class CustomBert_SelfReport(torch.nn.Module):
    def __init__(self):
        super(CustomBert_SelfReport, self).__init__()
        self.bert = bert_model
    
    def forward(self, ids_self_report, mask_self_report):
        output_encoding = self.bert(ids_self_report, mask_self_report)
        
        cls = output_encoding['last_hidden_state'].permute(1,0,2)[0]
        
        return cls

In [70]:
class CustomBert_Dialog(torch.nn.Module):
    def __init__(self):
        super( CustomBert_Dialog, self).__init__()
        self.bert = bert_model
    
    def forward(self, ids_dialog, mask_dialog):
        output_encoding = self.bert(ids_dialog, mask_dialog)
        
        cls = output_encoding['last_hidden_state'].permute(1,0,2)[0]
        
        return cls

In [71]:
class GAT_joint(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, args):
        super(GAT_joint, self).__init__()
        
        self.conv1 = GATConv(input_dim, hidden_dim, heads = args['heads'])
        self.conv2 = GATConv(args['heads']*hidden_dim, hidden_dim, heads = args['heads'])
        
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        
        x = self.conv1(x, edge_index)
        x = F.dropout(F.relu(x), p=args['dropout'], training = self.training)
        
        x = self.conv2(x, edge_index)
        x = F.dropout(F.relu(x), p=args['dropout'], training = self.training)
        
        x = global_mean_pool(x, batch)
        x = F.dropout(F.relu(x), p=args['dropout'], training = self.training)
        
        return x
        

In [72]:
768 + 384 

1152

In [73]:
768 + 768 + 384

1920

In [74]:
class Attention(torch.nn.Module):
  def __init__(self, hidden_dim1, hidden_dim2, proj_dim):
    super(Attention, self).__init__()

    self.W = torch.nn.Linear(hidden_dim1, proj_dim, bias=False)
    self.U = torch.nn.Linear(hidden_dim2, proj_dim, bias=False)
    self.V = torch.nn.Linear(proj_dim,1, bias=False)

  def forward(self, k, q):
      # k shape = [batch, 2, 768]
      # q shape = [batch, 384]
      # print("Q shape : ", q.shape)
      x = torch.unsqueeze(q, dim = 1)
      # print("Q shape : ", x.shape)
      out = F.tanh(self.W(k) + self.U(x))
      # print("Out shape : ", out.shape)
      out2 = self.V(out)
      # print("Out2 shape : ", out2.shape)
      out2 = torch.squeeze(out2, dim=-1)
      # print("Out2 shape : ", out2.shape)
      attn = F.softmax(out2, dim=-1)
      # print("Attention shape : ", attn.shape)
      attn = torch.unsqueeze(attn, dim=1)
      # print("Attention shape : ", attn.shape)
      context = torch.bmm(attn, k)
      # print("Context shape : ", context.shape)
      context = torch.squeeze(context, dim=1)
      # print("Context shape : ", context.shape)

      return context




In [75]:
class Linear_Knowledge_SelfReport(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, args, hidden_dim1, hidden_dim2, proj_dim):
        super(Linear_Knowledge_SelfReport, self).__init__()
        
        self.custom_bert_self_report = CustomBert_SelfReport()
        self.custom_bert_dialog = CustomBert_Dialog()
        self.gat_joint = GAT_joint(input_dim, hidden_dim, args)
        # self.linear1 = torch.nn.Linear(1920, 384)
        # self.linear2 = torch.nn.Linear(384, output_dim)
        # self.temp_linear = torch.nn.Linear(1920, output_dim)

        self.attention = Attention(hidden_dim1, hidden_dim2, proj_dim)
        self.temp_linear = torch.nn.Linear(1152, output_dim)
        
        

    def forward(self, data, input_ids_dialog, attention_mask_dialog, input_ids_self_report, attention_mask_self_report):
        bert_output_self_report = self.custom_bert_self_report(input_ids_self_report, attention_mask_self_report)
        bert_output_dialog = self.custom_bert_dialog(input_ids_dialog, attention_mask_dialog)
        gat_output = self.gat_joint(data)

        # print("Bert output self report : ", bert_output_self_report.shape)
        bert_output_self_report = torch.unsqueeze(bert_output_self_report, dim=1)
        # print("Bert output self report : ", bert_output_self_report.shape)

        # print("Bert output dialog : ", bert_output_dialog.shape)
        bert_output_dialog = torch.unsqueeze(bert_output_dialog, dim = 1)
        # print("Bert output dialog : ", bert_output_dialog.shape)

        bert_concat = torch.concat((bert_output_self_report, bert_output_dialog), dim=1)

        # print("Bert concat shape : ", bert_concat.shape)

        context = self.attention(bert_concat, gat_output)

        # print("Context shape : ", context.shape)
        
        final = torch.cat((context, gat_output), dim=1)


        
        # final = self.linear1(final)
        # final = F.relu(final)
        
        # final = self.linear2(final)

        final = self.temp_linear(final)
        
        return final

In [76]:
args = {
    'heads' : 3,
    'dropout' : 0.5
}

In [77]:
model = Linear_Knowledge_SelfReport(input_dim = 768, hidden_dim = 128, output_dim = 90, args = args, hidden_dim1 = 768, hidden_dim2 = 384, proj_dim = 64)
model

Linear_Knowledge_SelfReport(
  (custom_bert_self_report): CustomBert_SelfReport(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30526, 768)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768,

In [78]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [79]:
model = model.to(device)
model

Linear_Knowledge_SelfReport(
  (custom_bert_self_report): CustomBert_SelfReport(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30526, 768)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768,

In [80]:
model.parameters()

<generator object Module.parameters at 0x7f5f993dff90>

In [81]:
total_parameters = sum(p.numel() for p in model.parameters())
total_parameters/1e6

110.107546

In [82]:
total_train_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_train_parameters/1e6

110.107546

In [83]:
for i,name in enumerate(model.named_parameters()):
    print(i," : ",name[0])

0  :  custom_bert_self_report.bert.embeddings.word_embeddings.weight
1  :  custom_bert_self_report.bert.embeddings.position_embeddings.weight
2  :  custom_bert_self_report.bert.embeddings.token_type_embeddings.weight
3  :  custom_bert_self_report.bert.embeddings.LayerNorm.weight
4  :  custom_bert_self_report.bert.embeddings.LayerNorm.bias
5  :  custom_bert_self_report.bert.encoder.layer.0.attention.self.query.weight
6  :  custom_bert_self_report.bert.encoder.layer.0.attention.self.query.bias
7  :  custom_bert_self_report.bert.encoder.layer.0.attention.self.key.weight
8  :  custom_bert_self_report.bert.encoder.layer.0.attention.self.key.bias
9  :  custom_bert_self_report.bert.encoder.layer.0.attention.self.value.weight
10  :  custom_bert_self_report.bert.encoder.layer.0.attention.self.value.bias
11  :  custom_bert_self_report.bert.encoder.layer.0.attention.output.dense.weight
12  :  custom_bert_self_report.bert.encoder.layer.0.attention.output.dense.bias
13  :  custom_bert_self_report.b

In [84]:
for p in model.parameters():
    p.requires_grad = False

In [85]:
total_train_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_train_parameters/1e6

0.0

In [86]:
cnt=0
for name, param in model.named_parameters():
    if(param.requires_grad == False and cnt>=199):
        param.requires_grad = True
        print(name)
#         print()
    cnt+=1    

gat_joint.conv1.att_src
gat_joint.conv1.att_dst
gat_joint.conv1.bias
gat_joint.conv1.lin_src.weight
gat_joint.conv2.att_src
gat_joint.conv2.att_dst
gat_joint.conv2.bias
gat_joint.conv2.lin_src.weight
attention.W.weight
attention.U.weight
attention.V.weight
temp_linear.weight
temp_linear.bias


In [87]:
total_train_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_train_parameters/1e6

0.622234

In [88]:
1e-3

0.001

In [89]:
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
criterion = torch.nn.CrossEntropyLoss()


In [90]:
def train(loader):
    model.train()
    
    for data in loader:
        optimizer.zero_grad()
        
        graph_data = data[0]
        input_ids_dialog = data[1]
        attention_mask_dialog = data[2]
        input_ids_self_report = data[3]
        attention_mask_self_report = data[4]
        
        graph_data = graph_data.to(device)
        input_ids_dialog = input_ids_dialog.to(device)
        attention_mask_dialog = attention_mask_dialog.to(device)
        input_ids_self_report = input_ids_self_report.to(device)
        attention_mask_self_report = attention_mask_self_report.to(device)
        
        out = model(graph_data, input_ids_dialog, attention_mask_dialog, input_ids_self_report, attention_mask_self_report)
        
        z = graph_data.y
        ground_truth = z.long()
        
        loss = criterion(out, ground_truth)
        
        loss.backward()
        optimizer.step()

In [91]:
def test(loader):
    model.eval()

    y_true = []
    y_pred_top1 = []
    y_pred_top3 = []
    y_pred_top5 = []
    
    correct = 0
    for data in loader:
        with torch.no_grad():
            
            graph_data = data[0]
            input_ids_dialog = data[1]
            attention_mask_dialog = data[2]
            input_ids_self_report = data[3]
            attention_mask_self_report = data[4]

            graph_data = graph_data.to(device)
            input_ids_dialog = input_ids_dialog.to(device)
            attention_mask_dialog = attention_mask_dialog.to(device)
            input_ids_self_report = input_ids_self_report.to(device)
            attention_mask_self_report = attention_mask_self_report.to(device)

            out = model(graph_data, input_ids_dialog, attention_mask_dialog, input_ids_self_report, attention_mask_self_report)

            
            pred_top1 = out.argmax(dim=1)

            _, pred_top3 = torch.topk(out, 3, dim=1)
            _, pred_top5 = torch.topk(out, 5, dim=1)
            
            correct += int((pred_top1 == graph_data.y).sum())

            true_label = graph_data.y

            y_true.extend(true_label.tolist())
            y_pred_top1.extend(pred_top1.tolist())
            y_pred_top3.extend(pred_top3.tolist())
            y_pred_top5.extend(pred_top5.tolist())

            
    return correct/len(loader.dataset), y_true, y_pred_top1, y_pred_top3, y_pred_top5        

In [92]:
q1=torch.tensor([1,2,3,4,5])
q1

tensor([1, 2, 3, 4, 5])

In [93]:
q2=torch.tensor([2,2,2,4,6])
q2

tensor([2, 2, 2, 4, 6])

In [94]:
int((q1 == q2).sum())

2

In [95]:
768+384

1152

In [96]:
def train_and_validation(model, train_loader, valid_loader):
  best_acc = 0.0
  

  for epoch in range(25):

    print("\n=============Epoch : ", epoch)
    train( train_loader)
    # valid_acc, valid_pred, valid_gold = test(valid_loader)
    valid_acc, y_true_valid, y_pred_valid_top1, y_pred_valid_top3, y_pred_valid_top5= test(valid_loader)

    print("Length of predictions : ", len(y_pred_valid_top1))
    print("Length of gold : ", len(y_true_valid))
    # print("Valid loss : ", valid_loss)
    
    print("\nAccuracy\n", accuracy_score(y_true_valid, y_pred_valid_top1))
    print("\nF1 score\n", f1_score(y_true_valid, y_pred_valid_top1, average = 'macro'))
    print("\nJaccard similarity\n", jaccard_score(y_true_valid, y_pred_valid_top1, average = 'macro')) 

    curr_acc = accuracy_score(y_true_valid, y_pred_valid_top1)

    print('Curr acc : ', curr_acc)
    print('Best acc : ', best_acc)

    if(curr_acc > best_acc):
      best_acc = curr_acc

      
      torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/pakdd_baseline/saved_model/models/dialog_self_report_knowledge/two_disease/best_model_epoch_'+str(epoch)+'_best_acc_'+str(int(best_acc*100))+'_.pt')

      print("model saved\n")

In [97]:
train_and_validation(model, train_loader, valid_loader)


=============Epoch :  0


/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:1956: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Length of predictions :  137
Length of gold :  137

Accuracy
 0.31386861313868614

F1 score
 0.17420542420542423

Jaccard similarity
 0.14295891690009335
Curr acc :  0.31386861313868614
Best acc :  0.0
model saved


=============Epoch :  1


/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:1956: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Length of predictions :  137
Length of gold :  137

Accuracy
 0.5401459854014599

F1 score
 0.3843809523809524

Jaccard similarity
 0.3361269841269841
Curr acc :  0.5401459854014599
Best acc :  0.31386861313868614
model saved


=============Epoch :  2


...

Length of predictions :  137
Length of gold :  137

Accuracy
 0.6642335766423357

F1 score
 0.5567669172932331

Jaccard similarity
 0.4929824561403509
Curr acc :  0.6642335766423357
Best acc :  0.7007299270072993

=============Epoch :  23


/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:1956: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Length of predictions :  137
Length of gold :  137

Accuracy
 0.6496350364963503

F1 score
 0.5251502152817942

Jaccard similarity
 0.46571637426900586
Curr acc :  0.6496350364963503
Best acc :  0.7007299270072993

=============Epoch :  24


/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:1956: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Length of predictions :  137
Length of gold :  137

Accuracy
 0.656934306569343

F1 score
 0.5515064935064935

Jaccard similarity
 0.49139682539682544
Curr acc :  0.656934306569343
Best acc :  0.7007299270072993


In [101]:
# for epoch in range(25):
#     train(train_loader)
#     train_acc, y_true_train, y_pred_train_top1, y_pred_train_top3, y_pred_train_top5  = test(train_loader)
#     valid_acc, y_true_valid, y_pred_valid_top1, y_pred_valid_top3, y_pred_valid_top5 = test(valid_loader)
#     print(f'Epoch : {epoch : 03d}, Train acc : {train_acc : .4f}, Valid acc : {valid_acc : .4f}')

In [98]:
path = '/content/drive/MyDrive/Colab Notebooks/pakdd_baseline/saved_model/models/dialog_self_report_knowledge/two_disease/best_model_epoch_10_best_acc_70_.pt'

In [99]:
model.load_state_dict(torch.load(path))

<All keys matched successfully>

In [100]:
test_acc, y_true_test, y_pred_test_top1, y_pred_test_top3, y_pred_test_top5 = test(test_loader)
print("Test acc : ", test_acc)

/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:1956: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Test acc :  0.6556776556776557


In [101]:
print("=========Using Sklearn======")
print("\nConfusion matrix\n", confusion_matrix(y_true_test, y_pred_test_top1))
print("\nAccuracy\n", accuracy_score(y_true_test, y_pred_test_top1))
print("\nF1 score\n", f1_score(y_true_test, y_pred_test_top1, average = 'macro'))
print("\nJaccard similarity\n", jaccard_score(y_true_test, y_pred_test_top1, average = 'macro'))

=========Using Sklearn======

Confusion matrix
 [[0 0 2 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 2 ... 0 0 0]
 ...
 [0 0 0 ... 3 0 0]
 [0 0 0 ... 0 3 0]
 [0 0 0 ... 0 0 2]]

Accuracy
 0.6556776556776557

F1 score
 0.6234643197711379

Jaccard similarity
 0.5346533201646838


In [ ]:
with open("/content/drive/MyDrive/Colab Notebooks/27/self_report_full_dialog_joint_graph/having_first_utterance/ground_truth_test_two_disease_top.p", "wb") as f:
  pickle.dump(y_true_test, f)

In [ ]:
with open("/content/drive/MyDrive/Colab Notebooks/27/self_report_full_dialog_joint_graph/having_first_utterance/predicted_test_two_disease_top1.p", "wb") as f:
  pickle.dump(y_pred_test_top1, f)

In [ ]:
with open("/content/drive/MyDrive/Colab Notebooks/27/self_report_full_dialog_joint_graph/having_first_utterance/predicted_test_two_disease_top3.p", "wb") as f:
  pickle.dump(y_pred_test_top3, f)

In [ ]:
with open("/content/drive/MyDrive/Colab Notebooks/27/self_report_full_dialog_joint_graph/having_first_utterance/predicted_test_two_disease_top5.p", "wb") as f:
  pickle.dump(y_pred_test_top5, f)

In [ ]:
with open("/content/drive/MyDrive/Colab Notebooks/27/self_report_full_dialog_joint_graph/having_first_utterance/confusion_matrix_two_disease_top.p", "wb") as f:
  pickle.dump(confusion_matrix, f)

In [ ]:
confusion_matrix

<function sklearn.metrics._classification.confusion_matrix(y_true, y_pred, *, labels=None, sample_weight=None, normalize=None)>